<a href="https://colab.research.google.com/github/jumbokh/nknu-class/blob/main/NLP/notebooks/12_03a_LSTM_IMDB_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自然語言處理(NLP)應用

## 程式參考來源：
- https://keras.io/examples/nlp/text_classification_from_scratch/


## 影評(IMDB movie review)情緒分析 

### 資料集：https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
### 解壓縮後，刪除 train\unsup 目錄 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ln -fs /content/gdrive/My\ Drive /app

In [3]:
!tar -xzf /app/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# 檢查是否安裝成功
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

#define CUDNN_MAJOR 7
#define CUDNN_MINOR 5
#define CUDNN_PATCHLEVEL 0
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"


In [4]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar xfz aclImdb_v1.tar.gz
!rm -rf aclImdb/train/unsup

--2021-09-30 13:28:07--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  21.0MB/s    in 5.2s    

2021-09-30 13:28:13 (15.5 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## 建立訓練及驗證資料集

In [6]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

print(
    "Number of batches in raw_train_ds: %d"
    % tf.data.experimental.cardinality(raw_train_ds)
)
print(
    "Number of batches in raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds)
)
print(
    "Number of batches in raw_test_ds: %d"
    % tf.data.experimental.cardinality(raw_test_ds)
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


## 查看5筆資料內容

In [7]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

## 前置處理，刪除跳行標籤(br)

In [8]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

# 刪除跳行標籤(<br />)
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )


# 參數設定
max_features = 20000
embedding_dim = 128
sequence_length = 500

# TextVectorization 參考 https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization
# 1. 轉成小寫，去除標點符號 (lowercasing + punctuation stripping)
# 2. 分詞 (split each sample into words)
# 3. 重組為 ngrams
# 4. 將單字對應成整數 
# 5. 把樣本轉換為向量 
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# 去除 labels 欄位
text_ds = raw_train_ds.map(lambda x, y: x)
# 呼叫 adapt 函數建立字彙表
vectorize_layer.adapt(text_ds)


## 呼叫 vectorize_layer 轉換資料

In [9]:
# vectorize_layer 有兩種用法
# 1. 放進 model 內
'''
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = layers.Embedding(max_features + 1, embedding_dim)(x)
'''
# 2. 直接對 dataset 作轉換： 可非同步處理或使用GPU，獲得較佳的效能，以下採此作法。

def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# 呼叫 vectorize_layer 轉換資料
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# 呼叫 cache/prefetch，獲得較佳的效能
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

## 建立卷積模型(Conv1D)

In [10]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## 訓練模型

In [11]:
epochs = 3
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
625/625 [==============================] - 44s 20ms/step - loss: 0.5078 - accuracy: 0.7059 - val_loss: 0.3187 - val_accuracy: 0.8620
Epoch 2/3
625/625 [==============================] - 10s 15ms/step - loss: 0.2300 - accuracy: 0.9095 - val_loss: 0.3086 - val_accuracy: 0.8720
Epoch 3/3
625/625 [==============================] - 9s 15ms/step - loss: 0.1140 - accuracy: 0.9596 - val_loss: 0.3874 - val_accuracy: 0.8698


## 模型評估

In [12]:
print(model.evaluate(test_ds))

782/782 [==============================] - 9s 11ms/step - loss: 0.4066 - accuracy: 0.8616
[0.40662240982055664, 0.8615599870681763]


## 新資料預測

In [13]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# 建立一個模型，使用已訓練的權重(in outputs)
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

782/782 [==============================] - 10s 12ms/step - loss: 0.4066 - accuracy: 0.8616


[0.4066224694252014, 0.8615599870681763]

## Bidirectional LSTM on IMDB

## 建立LSTM模型

In [14]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,757,761
Trainable params: 2,757,761
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(train_ds, batch_size=32, epochs=2, validation_data=val_ds)

Epoch 1/2
625/625 [==============================] - 299s 468ms/step - loss: 0.5642 - accuracy: 0.7153 - val_loss: 0.5032 - val_accuracy: 0.7674
Epoch 2/2
625/625 [==============================] - 291s 465ms/step - loss: 0.4206 - accuracy: 0.8183 - val_loss: 0.3967 - val_accuracy: 0.8268


## 模型評估

In [16]:
print(model.evaluate(test_ds))

782/782 [==============================] - 138s 177ms/step - loss: 0.4466 - accuracy: 0.7953
[0.44662702083587646, 0.7953199744224548]


## 新資料預測

In [17]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# 建立一個模型，使用已訓練的權重(in outputs)
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

782/782 [==============================] - 147s 185ms/step - loss: 0.4466 - accuracy: 0.7953


[0.44662725925445557, 0.7953199744224548]